In [1]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'r8BGSocOtv6vZBnjcwkU5qM4u'
consumer_secret = 'CNAJg23IWXLvUUyJfuFUQOYfzvtKfQVQu6SsY82yJY7GGdhBDC'
access_token = '796083347315367936-grZPgV2cW8DvrozevvihPIJ1Oq9WQVK'
access_secret = 'oJoGFZYDDOXjyG30v0FBChWmGfF9RG6fXvDCVuNrDZJGb'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

--- Collecting positive tweets

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('positive.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print('&quot; Error on_data: %s&quot; % str(e)')
            return True
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())

### filter for positive tweets 
tracker1 = [':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=']
tracker2 = [':)', ':-)', '=)']
twitter_stream.filter(track=tracker2, languages=['fr'])

--- Collecting negative tweets

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('negative.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print('&quot; Error on_data: %s&quot; % str(e)')
            return True
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())

### filter for negative tweets
twitter_stream.filter(track=[':(', ':-(', '=(', ')-:', '):', ');', ')=', ":'(", ":'‑("], languages=['fr'])

--- Creating a clean file with positive tweets... 
1) only unique tweets
2) no ambigueus tweets with positive and negative smiles

In [1]:
import string
import json
smiles_positiv = [':)', ':-)', '=)']
smiles_negativ = [':(', ':-(', '=(', ')-:', '):', ');', ')=', ":'(", ":'‑("]

tweets_text = set()
f = open('pos_selected.json', 'a')

for line in open('positive.json'):
    try:
        new_tweet = json.loads(line)
        text = new_tweet['text']
        
        ### check for the repetitions
        len_old = len(tweets_text)
        tweets_text.add(text)
        len_new = len(tweets_text)
        repet_ok = len_new > len_old
        
        ### check for the presence of the positive and abscence of negative smiles 
        pos_smiles_found = [smile for smile in smiles_positiv if text.find(smile)>=0]
        neg_smiles_found = [smile for smile in smiles_negativ if text.find(smile)>=0]
        pos_ok = len(pos_smiles_found)>0
        neg_no = len(neg_smiles_found)==0
        
        if repet_ok and pos_ok and neg_no:
            f.write('"'+text+'"')
            f.write('\n')
    except:
        pass
f.close()

In [16]:
import string
import json
smiles_positiv = [':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p']
smiles_negativ = [':(', ':-(', '=(', ":'(", ":'‑("]

tweets_text = set()
f = open('neg_selected.json', 'w')

for line in open('negative.json'):
    try: 
        new_tweet = json.loads(line)
        text = new_tweet['text']
        
        ### check for the repetitions
        len_old = len(tweets_text)
        tweets_text.add(text)
        len_new = len(tweets_text)
        repet_ok = len_new > len_old
        
        ### check for the presence of the negative and abscence of positive smiles 
        pos_smiles_found = [smile for smile in smiles_positiv if text.find(smile)>=0]
        neg_smiles_found = [smile for smile in smiles_negativ if text.find(smile)>=0]
        pos_no = len(pos_smiles_found)==0
        neg_ok = len(neg_smiles_found)>0
        
        if repet_ok and pos_no and neg_ok:
            f.write('"'+text+'"')
            f.write('\n')
    except:
        pass
f.close()

--- Creating a clean file with negative tweets